# PureGym timetable

I find the timetable hard to read:
- the times are not aligned
- you have to click to see if there's some place available

I'll try to make it more convenient.

https://www.puregym.com/gyms/london-great-portland-street/timetable

## Get the time table data

In [1]:
from urllib import request, parse
from bs4 import BeautifulSoup

import json
import re

# parse javascript that contains the data (as a function parameter)
from slimit import ast
from slimit.parser import Parser
from slimit.visitors import nodevisitor

In [2]:
my_gym_name = 'London Great Portland Street'

all_gyms_url = 'https://www.puregym.com/gyms/'
timetable_url = 'https://www.puregym.com/api/gymclass/getclasses'
timetable_gym_id_param = 'gymId'

### Get the id of my gym

In [3]:
r = request.urlopen(all_gyms_url)
html = r.read().decode('utf-8')
html[:100]

'\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    \n\n<title>Gyms | PureGym</title>\n<link type="text/css" h'

In [4]:
soup = BeautifulSoup(html, 'html.parser')
script = soup.find_all('script', text=re.compile('.*"allGyms".*'))
tag = script[0]
script_text = tag.get_text()
script_text[:100]

'ReactDOM.render(React.createElement(Component.GymSearchListRoot, {"allGyms":[{"id":69,"url":"/gyms/a'

In [5]:
parser = Parser()
tree = parser.parse(script_text)

json_node = None
for node in nodevisitor.visit(tree):
    if isinstance(node, ast.Assign) and getattr(node.left, 'value', '') == '"allGyms"':
        json_node = node.right
        break

json_str = json_node.to_ecma()
json_str[:100]

'[{\n  "id": 69,\n  "url": "/gyms/aberdeen-kittybrewster/",\n  "urlName": "aberdeen-kittybrewster",\n  "n'

In [6]:
j = json.loads(json_str)
j[0]

{'id': 69,
 'joiningfee': 15.0,
 'latitude': 57.1614,
 'longitude': -2.1123,
 'monthlyfee': 19.99,
 'name': 'Aberdeen Kittybrewster',
 'postcode': 'AB24 3LJ',
 'status': 2,
 'streetAddress': 'Kittybrewster Retail Park, Bedford Road',
 'url': '/gyms/aberdeen-kittybrewster/',
 'urlName': 'aberdeen-kittybrewster'}

In [7]:
my_gym_id = None
for gym in j:
    if gym['name'] == my_gym_name:
        my_gym_id = gym['id']
        break

my_gym_id

106

### Get the time table of my gym

In [8]:
data = parse.urlencode({timetable_gym_id_param: my_gym_id})
url = timetable_url + '?' + data
json_str = request.urlopen(url).read().decode('utf-8')
json_str[:100]

'{\r\n  "filter": {\r\n    "availableClassTypes": [\r\n      "all",\r\n      "induction",\r\n      "class"\r\n   '

In [9]:
timetable_json = json.loads(json_str)
sessions = timetable_json['scheduledClasses']
sessions[0]

{'bookedCount': 20,
 'bookingId': 0,
 'canBeBooked': False,
 'classCapacity': 20,
 'classType': 'pure classes',
 'description': 'Legs, bums and tums – does exactly what it says. A class aimed at those areas we love to hate, helping to tone and work your core and lower body. The class can be completed by beginners through to experts as our instructors will push you according to your ability. Great if your goal is Weight Loss, Toning or General Fitness',
 'duration': 30,
 'entityId': 5603,
 'gymId': 106,
 'hexColor': None,
 'instructorName': 'Andrew Mukudzeu Tirivanhu',
 'isInduction': False,
 'name': 'Pure Legs, Bums & Tums',
 'reason': 2,
 'reasons': ['You must be logged in to Book a class'],
 'startDateTime': '2016-10-24T05:45:00Z',
 'studio': 'Studio',
 'waitingListCapacity': 10,
 'waitingListCount': 5}

## Display the timetable (WIP)

In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.DataFrame(sessions)
df.drop(['gymId', 'hexColor', 'reason', 'reasons'], axis=1, inplace=True)  # what is bookingId?
df['startDateTime'] = df['startDateTime'].apply(pd.to_datetime)
df.sort_values(by='startDateTime', axis=0, inplace=True)

df.head()

,bookedCount,bookingId,canBeBooked,classCapacity,classType,description,duration,entityId,instructorName,isInduction,name,startDateTime,studio,waitingListCapacity,waitingListCount
0,20,0,False,20,pure classes,"Legs, bums and tums – does exactly what it say...",30,5603,Andrew Mukudzeu Tirivanhu,False,"Pure Legs, Bums & Tums",2016-10-24 05:45:00,Studio,10,5
1,22,0,False,22,pure classes,Our cycle class is an indoor experience where ...,30,5616,Andrew Mukudzeu Tirivanhu,False,Pure Cycle,2016-10-24 06:25:00,Cycle Studio,10,9
2,20,0,False,20,pure classes,Spending as much time as possible in your opti...,30,5622,Andrew Mukudzeu Tirivanhu,False,Pure Fat burn,2016-10-24 07:00:00,Studio,10,9
3,0,0,False,3,induction,The induction is interactive so if you would l...,30,5635,Andrew Mukudzeu Tirivanhu,True,Weight Loss Induction,2016-10-24 07:45:00,Induction Point,0,0
4,18,0,False,18,pure classes,Increase your metabolism and boost your fitnes...,40,5648,Polyzoi Despoina,False,Pure Circuits,2016-10-24 11:10:00,Studio,10,10


In [12]:
df.dtypes

bookedCount                     int64
bookingId                       int64
canBeBooked                      bool
classCapacity                   int64
classType                      object
description                    object
duration                        int64
entityId                        int64
instructorName                 object
isInduction                      bool
name                           object
startDateTime          datetime64[ns]
studio                         object
waitingListCapacity             int64
waitingListCount                int64
dtype: object

In [13]:
df.describe()

,bookedCount,bookingId,classCapacity,duration,entityId,waitingListCapacity,waitingListCount
count,163.000000,163.0,163.000000,163.000000,163.000000,163.000000,163.000000
mean,6.858896,0.0,15.601227,36.871166,6422.564417,7.423313,2.177914
std,9.275534,0.0,7.712856,7.639361,769.920479,4.386984,3.872856
min,0.000000,0.0,3.000000,15.000000,5603.000000,0.000000,0.000000
25%,0.000000,0.0,3.000000,30.000000,5861.000000,0.000000,0.000000
50%,0.000000,0.0,20.000000,40.000000,6248.000000,10.000000,0.000000
75%,18.000000,0.0,21.000000,45.000000,6630.500000,10.000000,2.000000
max,22.000000,0.0,22.000000,45.000000,8831.000000,10.000000,10.000000


In [14]:
df.describe(include=[object, bool], exclude=[int])  # TODO: datetime

,canBeBooked,classType,description,instructorName,isInduction,name,studio
count,163,163,163,163,163,163,163
unique,1,2,20,12,2,20,3
top,False,pure classes,Our cycle class is an indoor experience where ...,Andrew Mukudzeu Tirivanhu,False,Pure Cycle,Studio
freq,163,121,28,42,121,28,93


In [15]:
# classType <-> isInduction ?
# name <-> description ?
# startDateTime: total number is less than unique number

In [16]:
tmp = df[['classCapacity', 'bookedCount', 'classType','name',  'startDateTime', 'duration', 'studio', 'instructorName']]
tmp[
    (tmp['startDateTime'] < pd.to_datetime('2016-10-29'))
    & (tmp['classCapacity'] > tmp['bookedCount'])
   ]

,classCapacity,bookedCount,classType,name,startDateTime,duration,studio,instructorName
3,3,0,induction,Weight Loss Induction,2016-10-24 07:45:00,30,Induction Point,Andrew Mukudzeu Tirivanhu
5,3,0,induction,Tone Induction,2016-10-24 11:10:00,30,Induction Point,Ross Price
8,3,1,induction,Strength & Conditioning Induction,2016-10-24 13:30:00,30,Induction Point,Samuel Roberts
15,18,17,pure classes,Pure Circuits,2016-10-24 19:00:00,45,Studio,Samuel Roberts
19,3,1,induction,General Fitness Induction,2016-10-25 07:45:00,30,Induction Point,Polyzoi Despoina
23,3,2,induction,Tone Induction,2016-10-25 13:30:00,30,Induction Point,Ross Price
25,3,1,induction,Weight Loss Induction,2016-10-25 16:30:00,30,Induction Point,Ross Price
28,3,1,induction,General Fitness Induction,2016-10-25 18:10:00,30,Induction Point,Andrew Mukudzeu Tirivanhu
35,10,1,induction,Tone Induction,2016-10-26 08:00:00,30,Induction Point,Polyzoi Despoina
39,3,1,induction,Weight Loss Induction,2016-10-26 13:30:00,30,Induction Point,Ross Price
